#### LeetCode - SQL - #197

Refer [197. Rising Temperature](https://leetcode.com/problems/rising-temperature/description/)

Write a solution to find all dates  `Id`  with higher temperatures compared to its previous dates (yesterday).

Return the result table in  **any order**.

In [ ]:
temperature_data = [[1, '2015-01-01', 10], [2, '2015-01-02', 25], [3, '2015-01-03', 20], [4, '2015-01-04', 30]]
temperature_columns = ['id', 'recordDate', 'temperature']
temperature_schema = {'id':'Int64', 'recordDate':'datetime64[ns]', 'temperature':'Int64'}

In [ ]:
import pandas

temperature_pandas_dataframe = pandas.DataFrame(data=temperature_data, columns=temperature_columns).astype(temperature_schema)

In [ ]:
from pyspark.sql import SparkSession

spark_context = SparkSession.builder.appName("LeetCode SQL").getOrCreate()

# Spark DataFrame using Pandas DataFrame
temperature_dataframe = spark_context.createDataFrame(temperature_pandas_dataframe)
temperature_dataframe.printSchema()
temperature_dataframe.show(5)

In [ ]:
from pyspark.sql.functions import col, date_sub, to_date

temperature_dataframe_joined \
    = temperature_dataframe \
        .alias("today") \
        .join(temperature_dataframe.alias("yesterday"), date_sub(to_date(col("today.recordDate")), 1) == to_date(col("yesterday.recordDate")), "inner") \
        .select(col("today.id").alias("today_id"), col("today.recordDate").alias("today_recordDate"), col("today.temperature").alias("today_temperature"), col("yesterday.temperature").alias("yesterday_temperature"))
temperature_dataframe_joined.show()

temperature_dataframe_joined \
    .filter(temperature_dataframe_joined.today_temperature > temperature_dataframe_joined.yesterday_temperature) \
    .select(temperature_dataframe_joined.today_id.alias("id")) \
    .show()